<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled12a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # Import pandas for data handling and plotting

# --- 1. Environmental & Global Parameters ---
class Environment:
    def __init__(self, total_sim_steps, days_per_cycle=100):
            self.total_sim_steps = total_sim_steps
                    self.days_per_cycle = days_per_cycle # Steps per "day" cycle
                            self.time = 0 # Current time step (index)

                                    # UV Profile (simplified sine wave for circadian cycle)
                                            # np.linspace creates an array with 'total_sim_steps' points, so indices 0 to total_sim_steps-1
                                                    self.uv_intensity = np.sin(np.linspace(0, 2 * np.pi * (total_sim_steps / days_per_cycle), total_sim_steps))
                                                            self.uv_intensity[self.uv_intensity < 0] = 0 # No negative UV, night is 0

                                                                def get_current_uv(self):
                                                                        # Ensure we don't go out of bounds at the very end of simulation
                                                                                if self.time >= self.total_sim_steps:
                                                                                            return 0.0 # Return 0 if simulation time somehow exceeds array bounds
                                                                                                    return self.uv_intensity[self.time]

                                                                                                        def step(self):
                                                                                                                self.time += 1
                                                                                                                        # Corrected: Use self.days_per_cycle as defined in __init__
                                                                                                                                self.current_day_step = self.time % self.days_per_cycle

                                                                                                                                # --- 2. Proto-Cell/System Class ---
                                                                                                                                class ProtoCell:
                                                                                                                                    def __init__(self, env):
                                                                                                                                            self.env = env

                                                                                                                                                    # Core Molecular Pools (as concentrations/amounts)
                                                                                                                                                            self.rna_pool = 100.0 # General RNA (templates, proto-rRNA)
                                                                                                                                                                    self.dna_pool = 10.0  # Basic DNA (more stable template)
                                                                                                                                                                            self.atp_pool = 50.0  # Energy currency
                                                                                                                                                                                    self.mcyw_peptide_pool = 5.0 # Foraging/Antenna peptides
                                                                                                                                                                                            self.other_peptide_pool = 2.0 # Non-mcyw peptides

                                                                                                                                                                                                    # Resource Stocks (internal to the proto-cell)
                                                                                                                                                                                                            self.internal_ntp_stock = 100.0
                                                                                                                                                                                                                    self.internal_aa_stock = 100.0
                                                                                                                                                                                                                            self.internal_trp_tyr_cys_stock = 10.0 # Foraged rare AAs

                                                                                                                                                                                                                                    # System States & Damage
                                                                                                                                                                                                                                            self.rna_damage_level = 0.0 # Accumulates from UV
                                                                                                                                                                                                                                                    self.protein_synthesis_active = True # Whether ribosomes are active
                                                                                                                                                                                                                                                            self.rna_dna_copying_active = True # Whether replication is active
                                                                                                                                                                                                                                                                    self.uv_safety_status = "SAFE" # Based on internal signals
                                                                                                                                                                                                                                                                            self.premature_stop_signal = 0.0 # Initialize here, updated in apply_uv_damage

                                                                                                                                                                                                                                                                                    # --- Parameters (tuned for clearer "Safety Reportage" behavior) ---
                                                                                                                                                                                                                                                                                            # UV Damage parameters
                                                                                                                                                                                                                                                                                                    self.uv_damage_rate_per_uv = 0.15 # INCREASE: Make UV more damaging (was 0.05)
                                                                                                                                                                                                                                                                                                            self.damage_tolerance_threshold = 4.0 # DECREASE: Make the threshold for arrest lower (was 5.0)

                                                                                                                                                                                                                                                                                                                    # Foraging parameters (mcyw efficiency, ATP cost, preference for rare AAs)
                                                                                                                                                                                                                                                                                                                            self.foraging_efficiency_mcyw = 0.12 # SLIGHTLY INCREASE: (was 0.1)
                                                                                                                                                                                                                                                                                                                                    self.foraging_atp_cost = 0.008 # DECREASE: Make foraging slightly cheaper (was 0.01)
                                                                                                                                                                                                                                                                                                                                            self.trp_tyr_cys_foraging_bias = 7.0 # INCREASE: Make foraging for rare AAs even more biased (was 5.0)

                                                                                                                                                                                                                                                                                                                                                    # Protein Synthesis parameters
                                                                                                                                                                                                                                                                                                                                                            self.protein_synthesis_rate = 0.1 # Keep as is
                                                                                                                                                                                                                                                                                                                                                                    self.protein_synthesis_ntp_cost = 0.1
                                                                                                                                                                                                                                                                                                                                                                            self.protein_synthesis_aa_cost = 0.1
                                                                                                                                                                                                                                                                                                                                                                                    self.protein_synthesis_atp_cost = 0.02 # Keep as is
                                                                                                                                                                                                                                                                                                                                                                                            self.mcyw_synthesis_bias = 0.4 # INCREASE: Prioritize making more mcyw (was 0.3)
                                                                                                                                                                                                                                                                                                                                                                                                                                   # Crucial for both ATP generation AND foraging for rare AAs

                                                                                                                                                                                                                                                                                                                                                                                                                                           # Stop Codon Precariousness/Signaling
                                                                                                                                                                                                                                                                                                                                                                                                                                                   self.precarious_codon_rate_per_damage = 0.2 # INCREASE: Make damage-induced stops more potent (was 0.1)
                                                                                                                                                                                                                                                                                                                                                                                                                                                           self.premature_stop_signal_threshold = 0.4 # DECREASE: Make the trigger for arrest more sensitive (was 0.5)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                   # ATP Synthesis (from mcyw antenna chains and UV)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                           self.atp_generation_rate_per_mcyw_uv = 0.03 # INCREASE: Make ATP generation more efficient (was 0.01)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               # Vital to prevent quick energy collapse

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       # Repair parameters
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               self.repair_rate = 0.1 # INCREASE: Make repair faster (was 0.05)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       self.repair_atp_cost = 0.01
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               self.trp_tyr_cys_for_repair_needed = 0.1 # Keep as is

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       # RNA/DNA Copying parameters
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               self.rna_dna_copy_rate = 0.02 # INCREASE: Allow for more significant growth (was 0.01)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       self.rna_dna_copy_ntp_cost = 0.05
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               self.rna_dna_copy_atp_cost = 0.01

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       # Degradation rates
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               self.rna_degradation_rate = 0.001
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       self.peptide_degradation_rate = 0.002
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               self.atp_degradation_rate = 0.005

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   # --- Core Processes ---
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       def _apply_uv_damage(self):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               current_uv = self.env.get_current_uv()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       if current_uv > 0:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   damage_increase = current_uv * self.uv_damage_rate_per_uv
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               self.rna_damage_level += damage_increase
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           # Simulate the "safety reportage" signal from damaged precarious codons
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       self.premature_stop_signal = damage_increase * self.precarious_codon_rate_per_damage
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               else:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           self.premature_stop_signal = 0 # No new stops at night

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               def _forage(self):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       if self.mcyw_peptide_pool <= 0 or self.atp_pool <= self.foraging_atp_cost:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   return # Cannot forage without mcyw or ATP

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           # Consume ATP for foraging movement
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   atp_consumed = self.foraging_atp_cost * self.mcyw_peptide_pool
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           self.atp_pool -= atp_consumed
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   self.atp_pool = max(0, self.atp_pool) # Ensure not negative

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           # Acquire general AAs and NTPs from the Environment's external pools
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   general_foraged = self.foraging_efficiency_mcyw * self.mcyw_peptide_pool
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           transfer_ntp = min(general_foraged / 2, self.env.external_ntp_pool) # Assume half NTP, half AA
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   transfer_aa = min(general_foraged / 2, self.env.external_aa_pool)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           self.internal_ntp_stock += transfer


IndentationError: unexpected indent (ipython-input-1-3638093710.py, line 9)